<a href="https://colab.research.google.com/github/cam2149/MachinelearningI/blob/main/Examen1_Ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maestría en Inteligencia Artificial Aplicada

## Aprendizaje Automatico 1

## Examen 1

### Integrantes

- Yesid Castelblanco
- Andres Felipe Borrero
- Carlos Alberto Martinez Ramirez
- Nicolas Colmenares

### **Profesores**

- Santiago Ortiz
- Henry Velasco

Fecha: _ de 2024

# Ejercicio 1

## Datos

Considere el conjunto de datos “data1” del fichero data_exam1.xls

### Puntos

1. Realice un análisis exploratorio de datos ¿Considera que podría generar un modelo de regresión lineal con variable categórica (sin interacción) para la variable Y? Justifique. Si la
respuesta a la pregunta es SI, genere un modelo de regresión sin interacción e interpretelo.

2. Realice un gráfico de dispersión para Y vs X, considerando para cada observación su respectivo valor en la variable Ind. ¿Hay evidencia muestral que sugiera un cambio en la tasa media de cambio de Y condicionado a incrementos unitarios de X? ¿Considera que un modelo con
interacciones sería más adecuado? Si la respuesta a estas preguntas es afirmativa, genere el
respectivo modelo, interprete detalladamente los resultados y valide los supuestos del modelo propuesto

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [3]:
url = 'https://github.com/cam2149/MachinelearningI/raw/85795f6469ebde1f693b7cb6c893c10135408974/data_exam1.xlsx'

df=pd.read_excel(url, sheet_name='data1')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# @title Correlation of X and Y


plt.figure(figsize=(8, 6))
plt.scatter(df['X'], df['Y'], c=df['Ind'], cmap='viridis')
plt.title('Correlation of X and Y')
plt.xlabel('X')
_ = plt.ylabel('Y')

In [ ]:
# @title Y vs X

df.plot(kind='scatter', x='Y', y='X', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title X

df['X'].plot(kind='hist', bins=20, title='X')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Y

df['Y'].plot(kind='hist', bins=20, title='Y')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
plt.boxplot(df['Y'])
plt.title('Boxplot of Y')
plt.ylabel('Y')
plt.show()


In [ ]:
# @title Histograms of X and Y

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.hist(df[df['Ind'] == 0]['X'], bins=20, alpha=0.5, label='X, Ind = 0', color='blue')
plt.hist(df[df['Ind'] == 1]['X'], bins=20, alpha=0.5, label='X, Ind = 1', color='green')
plt.hist(df[df['Ind'] == 0]['Y'], bins=20, alpha=0.5, label='Y, Ind = 0', color='red')
plt.hist(df[df['Ind'] == 1]['Y'], bins=20, alpha=0.5, label='Y, Ind = 1', color='purple')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of X and Y')
_ = plt.legend()

# Ejercicio 2

## Datos

**data2** del fichero **data_exam.xlsx**

## Puntos

- De acuerdo al analisis del item anterior proponer una transformación para alguna variable y justificar
- Proponer un modelo lineal, iterpretar y validar supuestos

# Ejercicio 3

## Datos

**Wine Quality** del fichero **datos.xlsx**

**Y**= Densidad

**Eliminar:**

- PH
- Sulfatos
- Cloruros
- Acidez Volátil
- Acidez Fija
- Calidad de Vino

## Puntos

1. Estandarizar variables y calcular matrices de correlación de **Pearson**, **Kendall** y **Spearman**. ¿Qué diferencia hay entre las dependencias obtenidas?
2. Realizar particios de datos 80-20.
  - Construir 3 modelos RLM.
  - Comparar e interpretar los valores de los coeficientes de regresión en cada método.
3. ¿Cuál de los modelos propuestos predice mejor?
  - Realizar una predicción con los datos de prueba.
  - Calcular el RMSE.
4. Validar supuesto teóricos de cada modelo.
5. Realizar un diagrama de dispersión del conjunto de datos.
  - ¿Se evidencia comportamientos lineales?
  - Si no son lineales realizar transformaciones de varaibles y justificar.
  - Generar un modelo RLM e interpretar.

In [13]:
# Get Excel file from GitHub
url = 'https://github.com/cam2149/MachinelearningI/blob/main/datos.xls?raw=true'
# Read the sheet 'Wine Quality' into a DataFrame df ignoring the first 2 rows
df = pd.read_excel(url, sheet_name='Wine Quality', skiprows=2)

In [ ]:
# Visualize the first 5 rows of the DataFrame
df.head()

In [ ]:
# Remove columns pH, Sulfatos, Cloruros, Acidez Volátil, Acidez Fija, and Calidad del Vino
df.drop(columns=['pH', 'Sulfatos', 'Cloruros', 'Acidez Volátil', 'Acidez Fija', 'Calidad del Vino'], inplace=True)

# Visualize the first 5 rows of the DataFrame
df.head()

In [ ]:
# Define Densidad as the target variable (Y) and the rest of the columns as the features (X)
x = df.drop(columns='Densidad')
y = df['Densidad']
x.describe()

In [ ]:
# Plot the scatter charts of the features against the target variable
fig, axs = plt.subplots(2, 3, figsize=(20, 10))
for i, feature in enumerate(x.columns):
    axs[i // 3, i % 3].scatter(x[feature], y, c='blue', alpha=0.5)
    axs[i // 3, i % 3].set_xlabel(feature)
    axs[i // 3, i % 3].set_ylabel('Densidad')
plt.tight_layout()
plt.show()



In [ ]:
# Standardize the features keeping the same column names
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
x.describe()


In [ ]:
# Plot the scatter charts of the features against the target variable
fig, axs = plt.subplots(2, 3, figsize=(20, 10))
for i, feature in enumerate(x.columns):
    axs[i // 3, i % 3].scatter(x[feature], y, c='blue', alpha=0.5)
    axs[i // 3, i % 3].set_xlabel(feature)
    axs[i // 3, i % 3].set_ylabel('Densidad')
plt.tight_layout()
plt.show()



In [ ]:
# Plot the Pearson correlation matrix of the features (X) and the target variable (Y) with a heatmap
corr = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Pearson')
plt.show()

# Now plot the Pearson correlation matrix of the standardized features (X) and the target variable (Y) with a heatmap
corr = x.join(y).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Pearson (Estandarizada)')
plt.show()




In [ ]:
# Plot the Kendall correlation matrix of the features (X) and the target variable (Y) with a heatmap
corr = df.corr(method='kendall')
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Kendall')
plt.show()

# Now plot the Kendall correlation matrix of the standardized features (X) and the target variable (Y) with a heatmap
corr = x.join(y).corr(method='kendall')
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Kendall (Estandarizada)')
plt.show()


# Ejercicio 4

## Datos

**publicidad.csv**

## Puntos

1. Graficar el retorno de la inversión **Sales** vs **TV, Radio, Newspaper**.
  - Usar scatter_matrix()
  - Interpretar graficos de las variables 2 a 2.
2. Calcular coeficiente de correlación.
  - Graficar mapa de calor de las correlaciones.
  - Interpretar las estructuras de dependencia encontradas.
3. Escoger una de las 3 variables para modelar las ventas.
  - Ajustar modelo de regresión lineal simple y encontrar la ecuación de la recta
  - ¿Cuál es el valor del coeficiente de determinación ${𝑅}^2$.
  - ¿Cómo se interpreta este valor?
4. Realizar una predicción del retorno a la inversión al realizar 5 anuncios.
  - ¿Cuál es el intervalo de confianza del 95% para la predicción?